# Session 3.4: BakeryAI - Advanced RAG with Agents

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1Smmawc1U6XkzS1Hl0b0QbkN6uRAW3Di9?usp=sharing)

## 🎯 The Ultimate Integration

Combine **everything** we've built:
- **Session 1**: LangChain basics
- **Session 2**: Agents, tools, LangGraph
- **Session 3**: RAG, vector stores, retrieval

### What We're Building:

```
Customer Query
     ↓
[Agent Reasoning]
     ↓
Choose Action:
├─ RAG Tool (policies)
├─ RAG Tool (products)
├─ RAG Tool (safety)
├─ Inventory Check
├─ Order Placement
└─ Calendar Tool
     ↓
[Synthesize Answer]
```

### BakeryAI Complete System:

**Can do everything:**
1. Answer policy questions (RAG)
2. Look up product details (RAG)
3. Check inventory (Tool)
4. Place orders (Tool)
5. Schedule delivery (Tool)
6. Remember conversation (Memory)
7. Multi-step reasoning (Agent)

Let's build the complete system! 🚀

In [1]:
!pip install -q langchain langchain-openai langchain-community langgraph
!pip install -q faiss-cpu chromadb
!pip install -q python-dotenv

In [2]:
!git clone https://github.com/IvanReznikov/mdx-langchain-conclave

fatal: destination path 'mdx-langchain-conclave' already exists and is not an empty directory.


In [3]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
import pandas as pd

import pickle
from google.colab import userdata
import os
from typing import List, TypedDict

# Set OpenAI API key from Google Colab's user environment or default
def set_openai_api_key(default_key: str = "YOUR_API_KEY") -> None:
    """Set the OpenAI API key from Google Colab's user environment or use a default value."""
    #if not (userdata.get("OPENAI_API_KEY") or "OPENAI_API_KEY" in os.environ):
    os.environ["OPENAI_API_KEY"] = userdata.get("MDX_OPENAI_API_KEY") or default_key


set_openai_api_key()
#set_openai_api_key("sk-...")

llm = ChatOpenAI(model="gpt-4o")
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

print("✅ Environment ready!")
print(f"Embedding model: text-embedding-3-small (1536 dimensions)")

✅ Environment ready!
Embedding model: text-embedding-3-small (1536 dimensions)


## 1. Load Knowledge Bases

Load vector stores from previous sessions.

In [4]:
# Load or create vector stores
try:
    vectorstore = FAISS.load_local(
        "/content/mdx-langchain-conclave/rag_artifacts/bakery_faiss_index",
        embeddings,
        allow_dangerous_deserialization=True
    )
    print("✅ Loaded vector store from disk")
except:
    print("⚠️  Creating sample vector store...")
    from langchain_core.documents import Document

    sample_docs = [
        Document(
            page_content="Refund Policy: Full refunds within 24 hours. After 24 hours, store credit only.",
            metadata={"source": "Customer_Service_Policy.txt", "category": "policy"}
        ),
        Document(
            page_content="Response Time: All inquiries must be answered within 2 hours during business hours (9 AM - 6 PM).",
            metadata={"source": "Customer_Service_Policy.txt", "category": "policy"}
        ),
        Document(
            page_content="Chocolate Truffle Cake: Rich Belgian chocolate. $45. Serves 8-10. Allergens: dairy, eggs, gluten.",
            metadata={"source": "cakes.pdf", "category": "product"}
        ),
        Document(
            page_content="Red Velvet Cake: Cream cheese frosting. $50. Serves 10-12. Allergens: dairy, eggs, gluten.",
            metadata={"source": "cakes.pdf", "category": "product"}
        ),
        Document(
            page_content="Vanilla Bean Cake: Madagascar vanilla. $40. Serves 8-10. Allergens: dairy, eggs, gluten.",
            metadata={"source": "cakes.pdf", "category": "product"}
        ),
        Document(
            page_content="Hand washing: Wash hands with soap for 20 seconds before food handling. Required before every shift.",
            metadata={"source": "SOP_Hygiene_Food_Safety.txt", "category": "safety"}
        ),
        Document(
            page_content="Food storage: Keep perishables below 40°F. Raw ingredients separate from finished products.",
            metadata={"source": "SOP_Hygiene_Food_Safety.txt", "category": "safety"}
        )
    ]

    vectorstore = FAISS.from_documents(sample_docs, embeddings)
    print("✅ Created sample vector store")

# Create category-specific retrievers
policy_retriever = vectorstore.as_retriever(
    search_kwargs={"k": 3, "filter": lambda m: m.get("category") == "policy"}
)
product_retriever = vectorstore.as_retriever(
    search_kwargs={"k": 3}
)
safety_retriever = vectorstore.as_retriever(
    search_kwargs={"k": 3, "filter": lambda m: m.get("category") == "safety"}
)

print("✅ Created specialized retrievers")

✅ Loaded vector store from disk
✅ Created specialized retrievers


In [5]:
# Load bakery data
try:
    cakes_df = pd.read_csv('/content/mdx-langchain-conclave/data/cake_descriptions.csv', encoding='cp1252')
    customers_df = pd.read_csv('/content/mdx-langchain-conclave/data/customers.csv', encoding='cp1252')
    orders_df = pd.read_csv('/content/mdx-langchain-conclave/data/orders.csv', encoding='cp1252')
    prices_df = pd.read_csv('/content/mdx-langchain-conclave/data/price_list.csv', encoding='utf-16')
    print(f"✅ Loaded: {len(cakes_df)} products, {len(customers_df)} customers, {len(orders_df)} orders")
except FileNotFoundError:
    cakes_df = pd.DataFrame({
        'Name': ['Chocolate Truffle Cake', 'Vanilla Bean Cake', 'Red Velvet Cake'],
        'Category': ['Chocolate', 'Vanilla', 'Specialty'],
        'Available': [True, True, True],
        'Weight_grams': [1000, 1000, 1200],
        'Delivery_time_hr': [24, 24, 48]
    })
    customers_df = pd.DataFrame({
        'client_id': ['C001', 'C002'],
        'client_name': ['Alice Johnson', 'Bob Smith'],
        'email': ['alice@example.com', 'bob@example.com'],
        'loyalty_points': [100, 50]
    })
    orders_df = pd.DataFrame({'order_id': ['O001'], 'status': ['delivered']})
    prices_df = pd.DataFrame({'Item': ['Chocolate Cake'], 'Unit_Price_AED': [45]})
    print("⚠️ Using sample data")

✅ Loaded: 22 products, 50 customers, 100 orders


## 2. Create RAG Tools

Wrap retrievers as tools that agents can use.

In [6]:
@tool
def search_policies(query: str) -> str:
    """Search customer service policies and company procedures.
    Use this for refund policies, response times, customer service standards.

    Args:
        query: The policy question to search for

    Returns:
        Relevant policy information
    """
    docs = vectorstore.similarity_search(query, k=2, filter={"category": "policy"})

    if not docs:
        return "No relevant policy information found."

    result = "📋 Policy Information:\n\n"
    for i, doc in enumerate(docs, 1):
        result += f"{i}. {doc.page_content}\n"
        result += f"   (Source: {doc.metadata.get('source', 'unknown')})\n\n"

    return result

@tool
def search_products(query: str) -> str:
    """Search product catalog for cakes, desserts, and their details.
    Use this for product information, prices, ingredients, allergens.

    Args:
        query: The product question to search for

    Returns:
        Relevant product information
    """
    docs = vectorstore.similarity_search(query, k=3, filter={"category": "product"})

    if not docs:
        return "No relevant product information found."

    result = "🍰 Product Information:\n\n"
    for i, doc in enumerate(docs, 1):
        result += f"{i}. {doc.page_content}\n\n"

    return result

@tool
def search_safety_procedures(query: str) -> str:
    """Search food safety and hygiene procedures.
    Use this for safety protocols, hygiene requirements, food handling.

    Args:
        query: The safety question to search for

    Returns:
        Relevant safety information
    """
    docs = vectorstore.similarity_search(query, k=2, filter={"category": "safety"})

    if not docs:
        return "No relevant safety information found."

    result = "🔒 Safety Procedures:\n\n"
    for i, doc in enumerate(docs, 1):
        result += f"{i}. {doc.page_content}\n"
        result += f"   (Source: {doc.metadata.get('source', 'unknown')})\n\n"

    return result

print("✅ Created RAG tools: search_policies, search_products, search_safety_procedures")

✅ Created RAG tools: search_policies, search_products, search_safety_procedures


## 3. Create Action Tools (from Session 2)

Add operational tools for orders, inventory, etc.

In [7]:
@tool
def check_inventory(product_name: str) -> str:
    """Check if a product is available in inventory.
    Use this to verify stock levels before confirming orders.

    Args:
        product_name: Name of the cake or product to check

    Returns:
        Availability status and details
    """
    product = cakes_df[cakes_df['Name'].str.contains(product_name, case=False, na=False)]

    if product.empty:
        return f"❌ Product '{product_name}' not found in our catalog."

    product_info = product.iloc[0]
    if product_info['Available']:
        return f"""✅ {product_info['Name']} is IN STOCK
              - Delivery time: {product_info['Delivery_time_hr']} hours
              - Category: {product_info['Category']}
              - Weight: {product_info['Weight_grams']}g"""
    else:
        return f"❌ {product_info['Name']} is OUT OF STOCK"

@tool
def calculate_order_total(product_name: str, quantity: int = 1) -> str:
    """Calculate the total cost including delivery fee.
    Use this to provide pricing information to customers.

    Args:
        product_name: Name of the product
        quantity: Number of items (default: 1)

    Returns:
        Total cost breakdown
    """
    try:
        filtered = prices_df[prices_df['Item'].str.lower().str.contains(product_name.lower(), na=False)]
        price = filtered['Unit_Price_AED'].iloc[0]
    except:
        price = 45  # default

    subtotal = price * quantity
    delivery_fee = 10 if subtotal < 100 else 0
    total = subtotal + delivery_fee

    return f"""💰 Order Total:
          - Product: {product_name} x {quantity}
          - Subtotal: {subtotal} AED
          - Delivery Fee: {delivery_fee} AED
          - TOTAL: {total} AED"""

@tool
def check_delivery_availability(date: str) -> str:
    """Check if delivery is available for a specific date.

    Args:
        date: Delivery date (e.g., 'tomorrow', '2025-10-25')

    Returns:
        Availability status
    """
    import random
    available = random.random() < 0.9

    if available:
        return f"✅ Delivery available for {date}. Available time slots: 10 AM - 12 PM, 2 PM - 4 PM, 4 PM - 6 PM."
    else:
        return f"❌ Delivery fully booked for {date}. Alternative dates available: tomorrow, day after."

print("✅ Created action tools: check_inventory, calculate_order_total, check_delivery_availability")

✅ Created action tools: check_inventory, calculate_order_total, check_delivery_availability


## 4. Build Complete Agent with RAG

Agent that can use both RAG and action tools.

In [8]:
# Collect all tools
all_tools = [
    search_policies,
    search_products,
    search_safety_procedures,
    check_inventory,
    calculate_order_total,
    check_delivery_availability
]

# Create system prompt
system_prompt = """
You are BakeryAI, an intelligent assistant for our bakery business.

You have access to:
1. RAG Tools - Search our knowledge base:
   - search_policies: Customer service policies, refund rules
   - search_products: Product catalog, prices, allergens
   - search_safety_procedures: Hygiene and food safety protocols

2. Action Tools - Perform operations:
   - check_inventory: Check stock levels
   - calculate_order_total: Calculate prices
   - check_delivery_availability: Check delivery slots

Guidelines:
- Use search tools to find accurate information
- Use action tools to check availability and calculate prices
- Always provide helpful, accurate responses
- Cite sources when using information from knowledge base
- If you don't know something, say so
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

# Create agent
agent = create_openai_functions_agent(
    llm=llm,
    tools=all_tools,
    prompt=prompt
)

# Create executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=all_tools,
    verbose=True,
    return_intermediate_steps=True,
    handle_parsing_errors=True
)

print("✅ Complete BakeryAI Agent with RAG created!")
print(f"   Total tools available: {len(all_tools)}")

✅ Complete BakeryAI Agent with RAG created!
   Total tools available: 6


## 5. Test: Knowledge Questions (RAG)

In [9]:
print("🧪 TEST 1: Knowledge Questions (RAG)")
print("=" * 70)

knowledge_questions = [
    "What is your refund policy?",
    "Tell me about chocolate cakes and their allergens",
    "What are the hand washing requirements?"
]

for q in knowledge_questions:
    print(f"\n❓ Question: {q}")
    result = agent_executor.invoke({"input": q})
    print(f"\n💡 Answer: {result['output']}")
    print("-" * 70)

🧪 TEST 1: Knowledge Questions (RAG)

❓ Question: What is your refund policy?


> Entering new AgentExecutor chain...

Invoking: `search_policies` with `{'query': 'refund policy'}`


📋 Policy Information:

1. Delivery Issues:  Track order status immediately  Communicate transparently about delays  If late 30 minutes: waive delivery fee  If wrong address (our error): deliver correct order  refund delivery  Follow up to ensure satisfaction Dissatisfied with Custom Order:  Discuss specific concerns  Offer modifications if time permits  If unable to resolve: full refund  keep cake (within reason)  Document for quality improvement  Manager approval required for refunds over AED 300  REFUND AND EXCHANGE POLICY: ELIGIBLE FOR FULL REFUND:  Product quality issues (stale, spoiled, incorrect)  Our error in order fulfillment  Delivery failure due to our fault  Within 24 hours of purchase with receipt NOT ELIGIBLE FOR REFUND:  Change of mind after custom order production begins  Allergic reactions (

## 6. Test: Action Questions (Tools)

In [10]:
print("\n🧪 TEST 2: Action Questions (Tools)")
print("=" * 70)

action_questions = [
    "Is chocolate truffle cake in stock?",
    "How much would 2 vanilla cakes cost?",
    "Can you deliver tomorrow?"
]

for q in action_questions:
    print(f"\n❓ Question: {q}")
    result = agent_executor.invoke({"input": q})
    print(f"\n💡 Answer: {result['output']}")
    print("-" * 70)


🧪 TEST 2: Action Questions (Tools)

❓ Question: Is chocolate truffle cake in stock?


> Entering new AgentExecutor chain...

Invoking: `check_inventory` with `{'product_name': 'chocolate truffle cake'}`


✅ Chocolate Truffle Cake is IN STOCK
              - Delivery time: 12 hours
              - Category: R
              - Weight: 1000gYes, the Chocolate Truffle Cake is in stock. It typically requires a 12-hour notice for delivery.

> Finished chain.

💡 Answer: Yes, the Chocolate Truffle Cake is in stock. It typically requires a 12-hour notice for delivery.
----------------------------------------------------------------------

❓ Question: How much would 2 vanilla cakes cost?


> Entering new AgentExecutor chain...

Invoking: `search_products` with `{'query': 'vanilla cake'}`


🍰 Product Information:

1. . Even though it leans into the richer side, the careful layering keeps each bite balanced. Its a birthday showpiece meant to be sharedbut you might want to save a slice for yourself

## 7. Test: Complex Multi-Step Questions

Questions that require both RAG and tools.

In [11]:
print("\n🧪 TEST 3: Complex Multi-Step Questions")
print("=" * 70)

complex_questions = [
    "I want to order a chocolate cake. Tell me about allergens, check if it's in stock, and calculate the price for 2 cakes.",
    "What's your refund policy? And if I order today, can you deliver tomorrow?",
    "Tell me about vanilla cake, check availability, and what's the response time if I have questions?"
]

for q in complex_questions:
    print(f"\n❓ Complex Question: {q}")
    print("\n🤖 Agent Processing...\n")
    result = agent_executor.invoke({"input": q})
    print(f"\n✨ Final Answer:\n{result['output']}")

    # Show which tools were used
    if result.get('intermediate_steps'):
        print("\n🔧 Tools Used:")
        tools_used = [step[0].tool for step in result['intermediate_steps']]
        for i, tool in enumerate(tools_used, 1):
            print(f"   {i}. {tool}")

    print("=" * 70)


🧪 TEST 3: Complex Multi-Step Questions

❓ Complex Question: I want to order a chocolate cake. Tell me about allergens, check if it's in stock, and calculate the price for 2 cakes.

🤖 Agent Processing...



> Entering new AgentExecutor chain...

Invoking: `search_products` with `{'query': 'chocolate cake'}`


🍰 Product Information:

1. . Rainbow Cake is light on complexity but big on impact. Its a birthday blockbuster. Contains eggs, gluten, dairy, and artificial colorants.  22. Chocolate Truffle Cake (R) A pure chocolate indulgence, this cake features dense chocolate layers filled and topped with rich truffle ganache. Its moist, rich, and designed for chocolate purists. Chocolate Truffle Cake is a celebration cake, but also an emotional support cakeone slice fixes everything. Best served slightly warm for that melt-in-your-mouth effect. Contains dairy, gluten, and soy.

2. . Its moist, rich, and designed for chocolate purists. Chocolate Truffle Cake is a celebration cake, but also an 

## 8. Query Routing with LangGraph

Intelligently route queries to appropriate knowledge bases.

In [12]:
from langgraph.graph import StateGraph, END
from typing import Annotated
from langchain_core.messages import BaseMessage

class QueryRouterState(TypedDict):
    """State for query routing"""
    query: str
    category: str  # 'policy', 'product', 'safety', 'action'
    result: str
    messages: List[BaseMessage]

def classify_query(state: QueryRouterState) -> QueryRouterState:
    """Classify query into category"""
    query = state['query'].lower()

    if any(word in query for word in ['refund', 'policy', 'return', 'complaint']):
        state['category'] = 'policy'
    elif any(word in query for word in ['cake', 'product', 'allergen', 'price', 'dessert']):
        state['category'] = 'product'
    elif any(word in query for word in ['hygiene', 'safety', 'wash', 'clean']):
        state['category'] = 'safety'
    else:
        state['category'] = 'action'

    print(f"📍 Classified as: {state['category']}")
    return state

def route_to_kb(state: QueryRouterState) -> str:
    """Route to appropriate knowledge base"""
    return state['category']

def search_policy_kb(state: QueryRouterState) -> QueryRouterState:
    """Search policy knowledge base"""
    result = search_policies.invoke({"query": state['query']})
    state['result'] = result
    return state

def search_product_kb(state: QueryRouterState) -> QueryRouterState:
    """Search product knowledge base"""
    result = search_products.invoke({"query": state['query']})
    state['result'] = result
    return state

def search_safety_kb(state: QueryRouterState) -> QueryRouterState:
    """Search safety knowledge base"""
    result = search_safety_procedures.invoke({"query": state['query']})
    state['result'] = result
    return state

def handle_action(state: QueryRouterState) -> QueryRouterState:
    """Handle action queries with agent"""
    result = agent_executor.invoke({"input": state['query']})
    state['result'] = result['output']
    return state

# Build routing graph
routing_graph = StateGraph(QueryRouterState)

# Add nodes
routing_graph.add_node("classify", classify_query)
routing_graph.add_node("policy", search_policy_kb)
routing_graph.add_node("product", search_product_kb)
routing_graph.add_node("safety", search_safety_kb)
routing_graph.add_node("action", handle_action)

# Set entry
routing_graph.set_entry_point("classify")

# Add routing
routing_graph.add_conditional_edges(
    "classify",
    route_to_kb,
    {
        "policy": "policy",
        "product": "product",
        "safety": "safety",
        "action": "action"
    }
)

# End edges
routing_graph.add_edge("policy", END)
routing_graph.add_edge("product", END)
routing_graph.add_edge("safety", END)
routing_graph.add_edge("action", END)

# Compile
router_app = routing_graph.compile()

print("✅ Query routing graph built!")

✅ Query routing graph built!


In [13]:
# Test query router
test_queries = [
    "What is the refund policy?",
    "Tell me about chocolate cake",
    "What are hand washing procedures?",
    "Check if vanilla cake is in stock"
]

print("🧪 TESTING QUERY ROUTER")
print("=" * 70)

for query in test_queries:
    print(f"\n❓ Query: {query}")

    result = router_app.invoke({
        "query": query,
        "category": "",
        "result": "",
        "messages": []
    })

    print(f"\n📚 Result:\n{result['result'][:200]}...")
    print("-" * 70)

🧪 TESTING QUERY ROUTER

❓ Query: What is the refund policy?
📍 Classified as: policy

📚 Result:
📋 Policy Information:

1. Delivery Issues:  Track order status immediately  Communicate transparently about delays  If late 30 minutes: waive delivery fee  If wrong address (our error): deliver correc...
----------------------------------------------------------------------

❓ Query: Tell me about chocolate cake
📍 Classified as: product

📚 Result:
🍰 Product Information:

1. . Its moist, rich, and designed for chocolate purists. Chocolate Truffle Cake is a celebration cake, but also an emotional support cakeone slice fixes everything. Best serve...
----------------------------------------------------------------------

❓ Query: What are hand washing procedures?
📍 Classified as: safety

📚 Result:
🔒 Safety Procedures:

1. .  PERSONAL HYGIENE REQUIREMENTS: 1. HANDWASHING PROTOCOL  Wash hands thoroughly for minimum 20 seconds with antibacterial soap  Required before: starting work, after breaks, .

## 9. Self-Querying Retriever

Let LLM construct structured queries with metadata filters.

In [14]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

# Define metadata schema
metadata_field_info = [
    AttributeInfo(
        name="category",
        description="The category of the document: 'policy', 'product', or 'safety'",
        type="string"
    ),
    AttributeInfo(
        name="source",
        description="The source file of the document",
        type="string"
    )
]

document_content_description = "Documents about bakery policies, products, and safety procedures"

# Create self-query retriever
try:
    self_query_retriever = SelfQueryRetriever.from_llm(
        llm=llm,
        vectorstore=vectorstore,
        document_contents=document_content_description,
        metadata_field_info=metadata_field_info,
        verbose=True
    )

    print("✅ Self-querying retriever created")

    # Test
    print("\n🧪 Testing self-query retriever:")

    # This should automatically filter by category
    results = self_query_retriever.invoke("Show me only product information about cakes")

    print(f"\n📚 Results: {len(results)} documents")
    for doc in results:
        print(f"   - {doc.page_content[:60]}... [Category: {doc.metadata.get('category')}]")

except Exception as e:
    print(f"⚠️  Self-query retriever requires specific FAISS setup: {e}")

⚠️  Self-query retriever requires specific FAISS setup: Self query retriever with Vector Store type <class 'langchain_community.vectorstores.faiss.FAISS'> not supported.


## 10. Complete BakeryAI System Demo

Full end-to-end demonstration.

In [15]:
class CompleteBakeryAI:
    """
    🎉 COMPLETE BAKERYAI SYSTEM

    Integrates:
    - RAG for knowledge retrieval
    - Agents for reasoning
    - Tools for actions
    - Memory for conversation
    - LangGraph for workflows
    """

    def __init__(self):
        self.agent = agent_executor
        self.router = router_app
        self.conversation_history = []

    def chat(self, message: str, use_router: bool = False):
        """Main chat interface"""

        if use_router:
            # Use query router
            result = self.router.invoke({
                "query": message,
                "category": "",
                "result": "",
                "messages": []
            })
            response = result['result']
        else:
            # Use full agent
            result = self.agent.invoke({
                "input": message,
                "chat_history": self.conversation_history
            })
            response = result['output']

        # Save to history
        self.conversation_history.append(HumanMessage(content=message))
        self.conversation_history.append(AIMessage(content=response))

        return response

    def clear_history(self):
        """Clear conversation history"""
        self.conversation_history = []

# Initialize BakeryAI
bakery_ai = CompleteBakeryAI()

print("🎉 COMPLETE BAKERYAI INITIALIZED!")
print("=" * 70)

🎉 COMPLETE BAKERYAI INITIALIZED!


In [16]:
# Complete conversation demo
print("\n💬 COMPLETE BAKERYAI CONVERSATION DEMO")
print("=" * 70)

conversation = [
    "Hi! I'm interested in ordering a cake.",
    "What chocolate cakes do you have?",
    "What are the allergens in the chocolate truffle cake?",
    "Is it in stock?",
    "How much for 2 cakes?",
    "What's your refund policy if I change my mind?"
]

for message in conversation:
    print(f"\n👤 Customer: {message}")
    response = bakery_ai.chat(message)
    print(f"🍰 BakeryAI: {response}")
    print("-" * 70)

print("\n✅ BakeryAI successfully handled complex multi-turn conversation!")
print("   - Used RAG for knowledge questions")
print("   - Used tools for inventory and pricing")
print("   - Maintained conversation context")
print("   - Combined multiple capabilities seamlessly")


💬 COMPLETE BAKERYAI CONVERSATION DEMO

👤 Customer: Hi! I'm interested in ordering a cake.


> Entering new AgentExecutor chain...
Great! I'd be happy to help you with that. Do you have a specific type or flavor of cake in mind, or would you like information about our available cakes? Additionally, how many people do you plan to serve, so we can recommend the right size?

> Finished chain.
🍰 BakeryAI: Great! I'd be happy to help you with that. Do you have a specific type or flavor of cake in mind, or would you like information about our available cakes? Additionally, how many people do you plan to serve, so we can recommend the right size?
----------------------------------------------------------------------

👤 Customer: What chocolate cakes do you have?


> Entering new AgentExecutor chain...

Invoking: `search_products` with `{'query': 'chocolate cakes'}`


🍰 Product Information:

1. . Rainbow Cake is light on complexity but big on impact. Its a birthday blockbuster. Contains eggs, 

## 🎯 Exercise 7: Build Multi-Document Question Answering

**Task**: Answer questions that require information from multiple documents:
1. Identify which documents are needed
2. Retrieve from multiple sources
3. Synthesize a cohesive answer
4. Cite all sources

In [17]:
def multi_document_qa(question: str):
    """Answer questions using multiple documents"""
    # TODO: Implement multi-document retrieval
    # Hint: Retrieve from multiple categories, synthesize answer
    pass

# Test with:
# "Can I order a chocolate cake for tomorrow, and what if it has allergens I can't have? What's your policy?"

## 🎯 Exercise 8: Implement Query Routing System

**Task**: Build intelligent query router:
1. Use LLM to classify query intent
2. Route to appropriate specialist agent
3. Handle edge cases (unclear queries)
4. Measure routing accuracy

In [18]:
class IntelligentRouter:
    def __init__(self):
        self.agents = {
            'policy': None,  # Policy specialist
            'product': None,  # Product specialist
            'operations': None  # Operations specialist
        }

    def route(self, query):
        """Route query to appropriate agent"""
        # TODO: Implement intelligent routing
        pass

# Build and test your router

## Summary: What We Built

### 🎉 Session 3.4 Achievements:

1. **RAG Tools**: Wrapped retrievers as agent tools
2. **Integrated Agent**: Combined RAG + actions
3. **Complex Reasoning**: Multi-step question handling
4. **Query Routing**: LangGraph-based intelligent routing
5. **Self-Query**: LLM-constructed metadata filters
6. **Complete System**: End-to-end BakeryAI

### ✨ Complete BakeryAI Capabilities:

**Knowledge Access (RAG):**
- ✅ Customer service policies
- ✅ Product catalog with details
- ✅ Safety and hygiene procedures

**Actions (Tools):**
- ✅ Check inventory
- ✅ Calculate pricing
- ✅ Verify delivery availability

**Intelligence (Agents):**
- ✅ Multi-step reasoning
- ✅ Tool selection
- ✅ Conversation memory
- ✅ Query routing

### 📊 Complete Course Progress:

```
Session 1: LangChain Foundation    [████████████████████] 100%
Session 2: Agents & LangGraph      [████████████████████] 100%
Session 3: RAG & Knowledge Bases   [████████████████████] 100%

Overall Progress: [██████████████░░░░░░] 75%
```

### 🎓 What Students Can Now Build:

After Session 3, students can build:
- ✅ Knowledge-base powered chatbots
- ✅ Multi-agent systems with RAG
- ✅ Intelligent query routing
- ✅ Complex reasoning over documents
- ✅ Production-ready Q&A systems

### 🚀 Next: Session 4

**Deployment & Observability:**
- LangServe for API deployment
- LangSmith for monitoring
- Production optimization
- Cost management
- Testing and evaluation

---

## 🏆 CONGRATULATIONS!

You've built a complete, production-ready AI system with:
- **Foundation**: LangChain components
- **Intelligence**: Agents and tools
- **Knowledge**: RAG pipelines
- **Integration**: Everything working together

**BakeryAI is now a sophisticated AI assistant capable of:**
- Answering complex questions
- Taking actions in the real world
- Learning from documents
- Reasoning across multiple steps

This same architecture can be adapted to ANY domain:
- Healthcare
- Finance
- Legal
- Education
- E-commerce

**You now have the skills to build production LLM applications!** 🚀